## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-20-13-00-00 +0000,missionlocal,El toro,https://missionlocal.org/2025/09/sf-photos-el-...
1,2025-09-20-12-57-03 +0000,nyt,Trump Says Critical Coverage of Him Is ‘Really...,https://www.nytimes.com/2025/09/19/us/politics...
2,2025-09-20-12-49-11 +0000,bbc,Lib Dems want 'Nightingale' courts to clear as...,https://www.bbc.com/news/articles/ckg6dd8207ro...
3,2025-09-20-12-46-55 +0000,startribune,Gun threats in greater Minnesota post-Annuncia...,https://www.startribune.com/gun-threats-in-gre...
4,2025-09-20-12-42-50 +0000,cbc,Large-scale Russian attack on Ukraine kills 3 ...,https://www.cbc.ca/news/world/russia-ukraine-d...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,66
129,new,24
35,kirk,18
54,kimmel,16
397,he,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
130,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...,183
260,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...,142
230,2025-09-19-18-27-14 +0000,cbc,Judge throws out Trump's $15B US lawsuit again...,https://www.cbc.ca/news/world/new-york-times-l...,134
279,2025-09-19-15-21-53 +0000,nypost,Trump says he told Charlie Kirk ‘you have a go...,https://nypost.com/2025/09/19/us-news/trump-sa...,134
226,2025-09-19-18-39-43 +0000,nyt,Judge Dismisses Trump’s $15 Billion Lawsuit Ag...,https://www.nytimes.com/2025/09/19/business/me...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
130,183,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...
99,90,2025-09-20-02-37-56 +0000,nypost,Bill Maher says Jimmy Kimmel was ‘wrong’ to su...,https://nypost.com/2025/09/19/media/bill-maher...
175,63,2025-09-19-21-31-00 +0000,wsj,Senate Democrats blocked Republicans’ stopgap ...,https://www.wsj.com/politics/policy/government...
126,41,2025-09-20-00-35-15 +0000,nypost,"Dem mayor, anti-ICE protesters hit with tear g...",https://nypost.com/2025/09/19/us-news/anti-ice...
282,39,2025-09-19-15-00-00 +0000,wsj,To teach her high-school students about econom...,https://www.wsj.com/us-news/education/how-teac...
199,39,2025-09-19-20-21-00 +0000,wsj,Trump administration officials are in discussi...,https://www.wsj.com/politics/national-security...
29,37,2025-09-20-10-47-20 +0000,bbc,Estonia seeks Nato consultation after Russian ...,https://www.bbc.com/news/articles/czrp6p5mj3zo...
260,37,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...
261,36,2025-09-19-16-24-28 +0000,latimes,Trump says China's Xi has approved a deal to s...,https://www.latimes.com/entertainment-arts/bus...
139,35,2025-09-19-23-43-00 +0000,wsj,Trump announced a move to dramatically reshape...,https://www.wsj.com/politics/policy/trump-to-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
